In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob

In [2]:
import os
filename = []
list = os.listdir('/Users/HP/Documents/Cap/capstone/j_prepared_data')
for f in list:
    if "all" in f:
        filename.append(f)

In [3]:
filename

['1751_all.csv',
 '1801_all.csv',
 '1810_all.csv',
 '1818_all.csv',
 '1933_all.csv']

In [4]:

df = pd.DataFrame()
for f in filename:
    df_c = pd.read_csv("../j_prepared_data/"+f)
    df=df.append(df_c)

    

In [5]:
features = df[['mxamp', 'std', 'skew', 'kurtosis', 'hr',  'patpk', 'patft']]
target = df[['sbp', 'dbp']]

In [6]:
features

,mxamp,std,skew,kurtosis,hr,patpk,patft
0,2879.176501,751.728982,-0.229849,-0.933357,88.669951,0.2675,0.152500
1,2765.073311,719.716591,-0.357917,-0.762510,83.916084,1.1150,0.156667
2,2833.671027,778.133839,-0.388241,-0.945357,90.909091,1.5975,0.166250
3,2783.069263,743.363892,-0.791137,-0.341928,95.490716,2.1600,0.168750
4,2804.659607,790.950755,-0.638073,-0.764912,97.826087,1.4950,0.165000
...,...,...,...,...,...,...,...
82,1648.349920,454.933806,-0.313119,-0.754156,98.901099,1.5075,0.160000
83,1659.651143,448.757452,-0.422786,-0.687299,98.630137,1.5100,0.167500
84,1610.583462,436.938508,-0.561483,-0.606359,100.558659,2.0850,0.172500
85,1718.082386,451.968913,-0.571665,-0.592175,104.347826,1.4500,0.171250


In [7]:
def cindex(true_labels, predictions):
    n,h = 0,0
    
    for i in range(1,len(true_labels)):
        t = true_labels[i]
        p = predictions[i]
        for j in range(i+1,len(true_labels)):
            nt = true_labels[j]
            npr = predictions[j]
            if t != nt:
                n += 1
                if (p < npr and t < nt) or (p > npr and t > nt):
                    h += 1
                else :
                    h += 0.5
                    
    return h/n

In [8]:
cindex([1,1,1,4,5,8], [1,1,1,5,4,8])


0.9444444444444444

In [17]:
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
loo = LeaveOneOut()
X = features.values
y = target.values
sbp_er = []
dbp_er = []
true_label = []
pred_label = []
acc_sbp = []
acc_dbp = []
for i, (train_idx, val_idx) in enumerate(loo.split(X)):
    X_train, X_test = X[train_idx], X[val_idx]
    y_train, y_test = y[train_idx], y[val_idx]
    
    model = make_pipeline(StandardScaler(),LinearRegression())
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    mae_sbp = mean_absolute_error(y_pred[:,0], y_test[:,0])
    mae_dbp = mean_absolute_error(y_pred[:,1], y_test[:,1])
    
    sbp_er.append(mae_sbp)
    dbp_er.append(mae_dbp)
    
    true_label.append(y_test)
    pred_label.append(y_pred)
    
    if mae_sbp<= 10:
        acc_sbp.append(y_pred[:,0])
    if mae_dbp <=10:
        acc_dbp.append(y_pred[:,1])      
    
true = [item for sublist in true_label for item in sublist]
pred = [item for sublist in pred_label for item in sublist]

true = np.array(true)
pred = np.array(pred)

cidx_sbp = cindex(true[:,0], pred[:,0])
cidx_dbp = cindex(true[:,1], pred[:,1])
    
print(f"Average MAE SBP: {np.mean(sbp_er)} MAE DBP: {np.mean(dbp_er)}")
print(f"C index SBP: {cidx_sbp}  C index DBP: {cidx_dbp}")
print(f"Probability SBP: {len(acc_sbp)/len(pred)}  Probability DBP: {len(acc_dbp)/len(pred)}" )

Average MAE SBP: 8.208646866302665 MAE DBP: 3.8478558600832264
C index SBP: 0.8406519726269409  C index DBP: 0.9019646449058653
Probability SBP: 0.6758620689655173  Probability DBP: 0.9195402298850575


In [18]:
df_j_lr = pd.DataFrame({
    "true_sbp": true[:,0], 
    "pred_sbp": pred[:,0],
    "true_dbp": true[:,1],
    "pred_dbp": pred[:,1]
                       })
df_j_lr

,true_sbp,pred_sbp,true_dbp,pred_dbp
0,94.670000,106.969426,81.235,77.266147
1,93.082000,108.594122,79.270,75.249239
2,91.200000,103.083400,77.075,71.415593
3,90.622000,100.994976,76.212,69.847957
4,90.712500,96.788248,76.080,71.764950
...,...,...,...,...
430,115.404000,109.442618,79.242,81.380522
431,113.977500,110.446345,78.740,78.021991
432,110.652000,109.318426,76.768,76.052854
433,109.262000,108.199708,75.214,76.664709


In [19]:
df_j_lr.to_csv("../Results/j_lr.csv", index = False)

In [10]:
from sklearn.ensemble import RandomForestRegressor

loo = LeaveOneOut()
X = features.values
y = target.values
sbp_er = []
dbp_er = []
true_label = []
pred_label = []
acc_sbp = []
acc_dbp = []
for i, (train_idx, val_idx) in enumerate(loo.split(X)):
    X_train, X_test = X[train_idx], X[val_idx]
    y_train, y_test = y[train_idx], y[val_idx]
    
    forest = make_pipeline(StandardScaler(),RandomForestRegressor(n_estimators = 200))
    forest.fit(X_train, y_train)
    y_pred = forest.predict(X_test)
    
    mae_sbp = mean_absolute_error(y_pred[:,0], y_test[:,0])
    mae_dbp = mean_absolute_error(y_pred[:,1], y_test[:,1])
    
    sbp_er.append(mae_sbp)
    dbp_er.append(mae_dbp)
    
    true_label.append(y_test)
    pred_label.append(y_pred)
    
    if mae_sbp<= 10:
        acc_sbp.append(y_pred[:,0])
    if mae_dbp <=10:
        acc_dbp.append(y_pred[:,1])   
        
true = [item for sublist in true_label for item in sublist]
pred = [item for sublist in pred_label for item in sublist]

true = np.array(true)
pred = np.array(pred)

cidx_sbp = cindex(true[:,0], pred[:,0])
cidx_dbp = cindex(true[:,1], pred[:,1])
    
print(f"Average MAE SBP: {np.mean(sbp_er)} MAE DBP: {np.mean(dbp_er)}")
print(f"C index SBP: {cidx_sbp}  C index DBP: {cidx_dbp}")
print(f"Probability SBP: {len(acc_sbp)/len(pred)}  Probability DBP: {len(acc_dbp)/len(pred)}" )

Average MAE SBP: 4.086566339080451 MAE DBP: 2.5548369386973184
C index SBP: 0.9294760592160577  C index DBP: 0.9344089569076531
Probability SBP: 0.9195402298850575  Probability DBP: 0.9770114942528736


In [11]:
df_j_rf = pd.DataFrame({
    "true_sbp": true[:,0], 
    "pred_sbp": pred[:,0],
    "true_dbp": true[:,1],
    "pred_dbp": pred[:,1]
                       })
df_j_rf

,true_sbp,pred_sbp,true_dbp,pred_dbp
0,94.670000,100.940735,81.235,77.193821
1,93.082000,113.703632,79.270,78.337991
2,91.200000,102.921800,77.075,76.532073
3,90.622000,95.474898,76.212,73.690912
4,90.712500,96.261863,76.080,74.569865
...,...,...,...,...
430,115.404000,114.522843,79.242,78.977031
431,113.977500,112.794117,78.740,77.749971
432,110.652000,110.946043,76.768,76.645571
433,109.262000,110.949497,75.214,76.618181


In [12]:
df_j_rf.to_csv("../Results/j_rf.csv", index = False)

In [13]:
from sklearn.svm import SVR
svr = make_pipeline(StandardScaler(),SVR(kernel = 'rbf', C=10, epsilon = 0.01))
X = features.values
y = target.values
sbp_er = []
dbp_er = []
true_label_sbp = []
pred_label_sbp = []

true_label_dbp = []
pred_label_dbp= []
acc_sbp = 0
acc_dbp = 0
for i, (train_idx, val_idx) in enumerate(loo.split(X)):
    X_train, X_test = X[train_idx], X[val_idx]
    y_train, y_test = y[train_idx], y[val_idx]
    

    svr.fit(X_train, y_train[:,0])
    y_pred_sbp = svr.predict(X_test)
    true_label_sbp.append(y_test[:,0])
    pred_label_sbp.append(y_pred_sbp)
    
    svr.fit(X_train, y_train[:,1])
    y_pred_dbp = svr.predict(X_test)
    true_label_dbp.append(y_test[:,1])
    pred_label_dbp.append(y_pred_dbp)
    
    mae_sbp = mean_absolute_error(y_pred_sbp, y_test[:,0])
    mae_dbp = mean_absolute_error(y_pred_dbp, y_test[:,1])
    
    sbp_er.append(mae_sbp)
    dbp_er.append(mae_dbp)
    
    if mae_sbp<= 10:
        acc_sbp = acc_sbp+1
    if mae_dbp <=10:
        acc_dbp = acc_dbp+1        
    
true_sbp = [item for sublist in true_label_sbp for item in sublist]
pred_sbp = [item for sublist in pred_label_sbp for item in sublist]

true_dbp = [item for sublist in true_label_dbp for item in sublist]
pred_dbp = [item for sublist in pred_label_dbp for item in sublist]

true_sbp = np.array(true_sbp)
pred_sbp = np.array(pred_sbp)

true_dbp = np.array(true_dbp)
pred_dbp = np.array(pred_dbp)

cidx_sbp = cindex(true_sbp, pred_sbp)
cidx_dbp = cindex(true_dbp, pred_dbp)
    
print(f"Average MAE SBP: {np.mean(sbp_er)} MAE DBP: {np.mean(dbp_er)}")
print(f"C index SBP: {cidx_sbp}  C index DBP: {cidx_dbp}")
print(f"Probability SBP: {acc_sbp/len(pred)}  Probability DBP: {acc_dbp/len(pred)}" )

Average MAE SBP: 5.202132262172269 MAE DBP: 2.8322151644157474
C index SBP: 0.907009291088856  C index DBP: 0.9275763348623365
Probability SBP: 0.8551724137931035  Probability DBP: 0.9655172413793104


In [15]:
df_j_svm = pd.DataFrame({
    "true_sbp": true_sbp, 
    "pred_sbp": pred_sbp,
    "true_dbp": true_dbp,
    "pred_dbp": pred_dbp
                       })
df_j_svm

,true_sbp,pred_sbp,true_dbp,pred_dbp
0,94.670000,110.543568,81.235,74.623103
1,93.082000,107.359981,79.270,75.036382
2,91.200000,105.498899,77.075,73.875390
3,90.622000,104.542743,76.212,73.661211
4,90.712500,95.699939,76.080,74.210251
...,...,...,...,...
430,115.404000,116.379036,79.242,81.972626
431,113.977500,112.768525,78.740,79.520144
432,110.652000,110.415824,76.768,78.022053
433,109.262000,111.128893,75.214,78.191094


In [16]:
df_j_svm.to_csv("../Results/j_svm.csv", index = False)